![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Multi-Source and Multi-Model AI Inference
This notebook is an example of how to perform AI inferences of multiple models processing multiple video streams.
Each video stream is fed to every model. Each model processes frames from every video stream in a multiplexing manner.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `inference_host_address` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

The script can use a web camera(s) or local camera(s) connected to the machine running this code or it can use video file(s).
The camera index or URL or video file path should be specified in the code below by assigning `video_sources`.

In [1]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

Name: degirum_tools
Version: 0.10.1
Summary: Tools for PySDK
Home-page: 
Author: DeGirum
Author-email: 
License: 
Location: c:\users\shashichilappagari\anaconda3\envs\supervision\lib\site-packages
Requires: degirum, ipython, numpy, opencv-python, pafy, pillow, psutil, pycocotools, python-dotenv, pyyaml, requests, scipy, youtube-dl
Required-by: 


#### Specify video sources and AI model names here

In [2]:
# inference_host_address: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# video_sources: list of video sources
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_names: list of AI models to use for inferences (NOTE: they should have the same input size)
# allow_frame_drop:
#     when True, we drop video frames in case when AI performance is not enough to work in real time
#     when False, we buffer video frames to keep up with AI performance
inference_host_address = "@cloud"
video_sources = [
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4",
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/TrafficHD.mp4",
]
model_zoo_url = "degirum/public"
model_names = [
    "yolo_v5s_hand_det--512x512_quant_n2x_orca1_1",
    "yolo_v5s_face_det--512x512_quant_n2x_orca1_1",
    "yolo_v5n_car_det--512x512_quant_n2x_orca1_1",
    "yolo_v5s_person_det--512x512_quant_n2x_orca1_1",
]
allow_frame_drop = False

#### Specify where do you want to run your inferences

In [3]:
import degirum as dg, degirum_tools

: 

In [4]:
from degirum_tools import streams as dgstreams

c = dgstreams.Composition()

batch_size = len(
    video_sources
)  # set AI server batch size equal to the # of video sources for lowest latency

# create PySDK AI model objects
models = []
for mi, model_name in enumerate(model_names):
    model = dg.load_model(
        model_name=model_name,
        inference_host_address=inference_host_address,
        zoo_url=model_zoo_url,
        token=degirum_tools.get_token(),
    )
    model.measure_time = True
    model.eager_batch_size = batch_size
    model.frame_queue_depth = batch_size
    models.append(model)

# check that all models have the same input configuration
models_have_same_input = True
for model in models[1:]:
    if (
        type(model._preprocessor) != type(models[0]._preprocessor)
        or model.model_info.InputH != models[0].model_info.InputH
        or model.model_info.InputW != models[0].model_info.InputW
    ):
        models_have_same_input = False

resizers = []

# create video sources and image resizers
# (we use separate resizers to do resize only once per source when possible, to improve performance),
# connect each resizer to corresponding video source
for src in video_sources:
    source = c.add(dgstreams.VideoSourceGizmo(src))
    if models_have_same_input:
        resizer = c.add(
            dgstreams.AiPreprocessGizmo(
                models[0], stream_depth=2, allow_drop=allow_frame_drop
            )
        )
    else:
        resizer = c.add(dgstreams.FanoutGizmo(allow_drop=allow_frame_drop))

    resizer.connect_to(source)  # connect resizer to video source
    resizers.append(resizer)

# create result combiner
combiner = c.add(dgstreams.AiResultCombiningGizmo(len(models)))

# create multi-input detector gizmos,
# connect each detector gizmo to every resizer gizmo,
# connect result combiner gizmo to each detector gizmo
for mi, model in enumerate(models):
    # create AI gizmo (aka detector) from the model
    detector = c.add(
        dgstreams.AiSimpleGizmo(model, stream_depth=2, inp_cnt=len(video_sources))
    )

    # connect detector gizmo to each resizer gizmo
    for fi, resizer in enumerate(resizers):
        detector.connect_to(resizer, fi)

    # connect result combiner gizmo to detector gizmo
    combiner.connect_to(detector, mi)

# create multi-window video multiplexing display gizmo
# and connect it to combiner gizmo
win_captions = [f"Stream #{i}: {str(src)}" for i, src in enumerate(video_sources)]
display = c.add(
    dgstreams.VideoDisplayGizmo(
        win_captions, show_ai_overlay=True, show_fps=True, multiplex=True
    )
)
display.connect_to(combiner)

# start composition
c.start()

Successfully opened video stream 'https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4'Successfully opened video stream 'https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/TrafficHD.mp4'



packet queue is empty, aborting
packet queue is empty, aborting
packet queue is empty, aborting
packet queue is empty, aborting
